<a href="https://colab.research.google.com/github/pavankumarchowdary35/Fake_news_classifier_LSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/fake-news (2)/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
###Drop Nan Values
df=df.dropna()


In [6]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.12.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2017, 3651, 284, 3303, 4425, 1187, 4020, 2123, 1352, 930],
 [1461, 1095, 2256, 3133, 2709, 184, 2516],
 [942, 348, 1490, 537],
 [3447, 4637, 324, 2211, 4589, 628],
 [2357, 2709, 2560, 3234, 1247, 1982, 2709, 273, 2415, 4217],
 [3129,
  1015,
  4626,
  4600,
  591,
  2574,
  2469,
  859,
  3405,
  1978,
  1184,
  2931,
  4293,
  4553,
  2516],
 [2356, 3346, 1802, 2281, 3119, 3857, 3020, 1052, 4630, 4580, 3114],
 [979, 2655, 3313, 3423, 229, 2313, 2574, 2863, 4630, 4580, 3114],
 [628, 719, 4307, 2030, 1082, 2468, 3193, 1485, 2574, 3163],
 [1090, 3282, 1379, 3797, 498, 4886, 3267, 2783],
 [3842, 2495, 3153, 715, 2708, 3470, 3384, 1568, 1370, 3879, 3529],
 [2211, 1651, 4425, 2468, 2574, 229],
 [3498, 4930, 768, 1, 1216, 3113, 451, 2483, 2217],
 [798, 3470, 1242, 2364, 2931, 3377, 975, 4630, 4580, 3114],
 [3141, 2765, 4461, 1336, 2504, 4630, 4580, 3114],
 [1389, 3349, 4858, 1834, 1169, 706, 4415, 2073, 462, 3508],
 [4979, 4114, 1095],
 [1472, 4281, 593, 2820, 2574, 3300, 3298, 2516],
 [48

### Embedding Representation

In [60]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2123 1352  930]
 [   0    0    0 ... 2709  184 2516]
 [   0    0    0 ...  348 1490  537]
 ...
 [   0    0    0 ... 4630 4580 3114]
 [   0    0    0 ... 4797 1019   95]
 [   0    0    0 ... 1270 3514  460]]


In [61]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2017,
       3651,  284, 3303, 4425, 1187, 4020, 2123, 1352,  930], dtype=int32)

In [62]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
len(embedded_docs),y.shape

(18285, (18285,))

In [64]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [65]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [67]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 33ms/step - loss: 0.3398 - accuracy: 0.8406 - val_loss: 0.1976 - val_accuracy: 0.9125
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1477 - accuracy: 0.9408 - val_loss: 0.2031 - val_accuracy: 0.9190
Epoch 3/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1070 - accuracy: 0.9596 - val_loss: 0.2269 - val_accuracy: 0.9148
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0781 - accuracy: 0.9706 - val_loss: 0.2790 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0519 - accuracy: 0.9820 - val_loss: 0.2975 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0342 - accuracy: 0.9883 - val_loss: 0.4121 - val_accuracy: 0.9042
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0234 - accuracy: 0.9931 - val_loss: 0.4102 - val_accuracy: 0.9175

### Adding Dropout

In [52]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [53]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [54]:
y_pred

array([[0.49953976],
       [0.50172627],
       [0.5010539 ],
       ...,
       [0.50154   ],
       [0.49752864],
       [0.50039876]], dtype=float32)

In [55]:
from sklearn.metrics import confusion_matrix

In [68]:
# Convert values below 0.5 to 0 and values >= 0.5 to 1
y_pred_binary = (y_pred <= 0.5).astype(int)

# Print the modified y_pred array
print(y_pred_binary)

[[1]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]


In [69]:
confusion_matrix(y_test,y_pred_binary)

array([[2292, 1127],
       [ 932, 1684]])

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_binary)

0.6588235294117647